In [6]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
# ==================== Helper Functions from Exercise 1 ====================
def k1(x0, x, w):
    """Kernel h1: rectangular window [0, 1)"""
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i] >= x0 and x[i] <= x0 + w:
            y[i] = 1
        else:
            y[i] = 0
    return y

def k2(x0, x, w):
    """Kernel h2: rectangular window centered [-1/2, 1/2)"""
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i] >= (x0 - (w/2)) and x[i] <= (x0 + (w/2)):
            y[i] = 1
        else:
            y[i] = 0
    return y

def k3(x0, x, w):
    """Kernel h3: linear (tent/triangle) [−1, 1]"""
    y = np.zeros(len(x))
    for i in range(len(x)):
        t = (x[i] - x0) / w
        if t <= -1 or t >= 1:
            y[i] = 0
        else:
            y[i] = 1 - np.abs(t)
    return y

def MSE(y, y_):
    """Calculate Mean Squared Error"""
    sum_val = 0
    for i in range(len(y)):
        sum_val += (y[i] - y_[i]) * (y[i] - y_[i])
    return sum_val / len(y)

# ==================== Image Scaling Functions ====================

def create_averaging_kernel(size):
    """Create averaging kernel Km for downscaling"""
    kernel = np.ones((size, size)) / (size * size)
    return kernel

def convolve2d_manual(image, kernel):
    """Manual 2D convolution implementation"""
    img_h, img_w = image.shape
    ker_h, ker_w = kernel.shape
    
    pad_h = ker_h // 2
    pad_w = ker_w // 2
    
    # Pad image
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='edge')
    
    # Output image
    output = np.zeros_like(image)
    
    for i in range(img_h):
        for j in range(img_w):
            region = padded[i:i+ker_h, j:j+ker_w]
            output[i, j] = np.sum(region * kernel)
    
    return output

def downsample_image(image, scale_factor):
    """
    Downsample image using averaging kernel
    
    Args:
        image: grayscale image (2D numpy array)
        scale_factor: integer factor to downsample by
    
    Returns:
        downsampled image
    """
    # Create averaging kernel
    kernel = create_averaging_kernel(scale_factor)
    
    # Apply convolution
    smoothed = convolve2d_manual(image, kernel)
    
    # Subsample (take every scale_factor-th pixel)
    downsampled = smoothed[::scale_factor, ::scale_factor]
    
    return downsampled

def interpolate_1d_row(row, kernel_type, scale):
    """Interpolate a single row using 1D interpolation"""
    x = np.arange(len(row))
    y = row
    
    x_new = np.zeros(scale * len(x))
    y_new = np.zeros(scale * len(x))
    k_table = np.zeros(scale * len(x))
    
    w = 1.0  # pixel spacing
    
    # Create new x coordinates
    krok = 0
    for i in range(0, len(x_new), scale):
        x_new[i] = x[krok]
        for j in range(0, scale - 1, 1):
            x_new[i + j + 1] = x[krok] + (j + 1) * (w / scale)
        krok += 1
    
    # Interpolate
    for i in range(len(x)):
        k_ = np.zeros(len(x_new))
        if kernel_type == "k1":
            k_ = k1(x[i], x_new, w)
        elif kernel_type == "k2":
            k_ = k2(x[i], x_new, w)
        elif kernel_type == "k3":
            k_ = k3(x[i], x_new, w)
        
        k_table += k_
        y_new += k_ * y[i]
    
    # Normalize
    for i in range(len(y_new)):
        if k_table[i] != 0:
            y_new[i] /= k_table[i]
    
    return y_new

def upsample_image(image, kernel_type, scale):
    """
    Upsample image using 1D interpolation applied separably
    
    Args:
        image: grayscale image (2D numpy array)
        kernel_type: "k1", "k2", or "k3"
        scale: integer factor to upsample by
    
    Returns:
        upsampled image
    """
    h, w = image.shape
    
    # First, interpolate along rows (horizontal)
    temp = np.zeros((h, w * scale))
    for i in range(h):
        temp[i, :] = interpolate_1d_row(image[i, :], kernel_type, scale)
    
    # Then, interpolate along columns (vertical)
    result = np.zeros((h * scale, w * scale))
    for j in range(w * scale):
        result[:, j] = interpolate_1d_row(temp[:, j], kernel_type, scale)
    
    return result

def scale_image_full_pipeline(image, down_factor, up_factor, kernel_type):
    """
    Complete pipeline: downsample then upsample
    
    Args:
        image: original grayscale image
        down_factor: factor to downsample by
        up_factor: factor to upsample by
        kernel_type: interpolation kernel ("k1", "k2", "k3")
    
    Returns:
        tuple: (downsampled, upsampled, mse_value)
    """
    # Downsample
    downsampled = downsample_image(image, down_factor)
    
    # Upsample
    upsampled = upsample_image(downsampled, kernel_type, up_factor)
    
    # Calculate MSE if dimensions match
    if upsampled.shape == image.shape:
        mse_value = MSE(image.flatten(), upsampled.flatten())
    else:
        mse_value = None
    
    return downsampled, upsampled, mse_value

# ==================== Demonstration ====================

# Load or create a test image
def create_test_image(size=64):
    """Create a simple test image: white background with black rectangle"""
    # White background
    img = np.ones((size, size)) * 255
    
    # Add black rectangle in the center
    rect_size = size // 3
    start = size // 3
    end = start + rect_size
    img[start:end, start:end] = 0
    
    return img

# Example usage
if __name__ == "__main__":
    # Create output folder
    import os
    output_folder = "scaling"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created folder: {output_folder}")
    # Create test image
    # Wczytaj obraz z internetu
    original = Image.open(r"C:\Users\Aktow\Desktop\Projekty\SIOC\obraz.png").convert("L")
    original = np.array(original)
    # Or load your own image:
    # original = np.array(Image.open('your_image.jpg').convert('L'))
    
    kernels = ["k1", "k2", "k3"]
    
    print("=" * 60)
    print("EXPERIMENT 1: Downscale only (test different factors)")
    print("=" * 60)
    
    down_factors = [2, 4]
    
    for factor in down_factors:
        downsampled = downsample_image(original, factor)
        
        # Save exact pixel dimensions
        h, w = downsampled.shape
        dpi = 100
        fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(downsampled, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
        plt.savefig(f'{output_folder}/downsampled_{factor}x.png', dpi=dpi)
        plt.close()
        
        print(f"Factor {factor}: {original.shape} → {downsampled.shape}")
    
    print("\n" + "=" * 60)
    print("EXPERIMENT 2: Upscale only (test different kernels)")
    print("=" * 60)
    
    # Show original first
    h, w = original.shape
    dpi = 100
    fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(original, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
    plt.savefig(f'{output_folder}/original.png', dpi=dpi)
    plt.close()
    
    up_factor = 2
    
    for kernel in kernels:
        upsampled = upsample_image(original, kernel, up_factor)
        
        # Save exact pixel dimensions
        h, w = upsampled.shape
        dpi = 100
        fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(upsampled, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
        plt.savefig(f'{output_folder}/upsampled_{kernel}_{up_factor}x.png', dpi=dpi)
        plt.close()
        
        print(f"Kernel {kernel}: {original.shape} → {upsampled.shape}")
    
    print("\n" + "=" * 60)
    print("EXPERIMENT 3: Full pipeline (down then up) - Quality test")
    print("=" * 60)
    
    # Test: downsample by 2, then upsample by 2 (should return to original size)
    down_factor = 2
    up_factor = 2
    
    for kernel in kernels:
        print(f"\nKernel {kernel}:")
        
        downsampled, upsampled, mse_val = scale_image_full_pipeline(
            original, down_factor, up_factor, kernel
        )
        
        print(f"  Original: {original.shape}")
        print(f"  After downsample: {downsampled.shape}")
        print(f"  After upsample: {upsampled.shape}")
        print(f"  MSE: {mse_val:.4f}")
        
        # Save images with exact pixel dimensions
        dpi = 100
        
        # Original
        h, w = original.shape
        fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(original, cmap='gray', vmin=0, vmax=255)
        plt.savefig(f'{output_folder}/pipeline_{kernel}_1_original.png', dpi=dpi)
        plt.close()
        
        # Downsampled
        h, w = downsampled.shape
        fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(downsampled, cmap='gray', vmin=0, vmax=255)
        plt.savefig(f'{output_folder}/pipeline_{kernel}_2_downsampled.png', dpi=dpi)
        plt.close()
        
        # Upsampled (reconstructed)
        h, w = upsampled.shape
        fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(upsampled, cmap='gray', vmin=0, vmax=255)
        plt.savefig(f'{output_folder}/pipeline_{kernel}_3_reconstructed.png', dpi=dpi)
        plt.close()
        
        # Error map (keep this one with colorbar for reference)
        if mse_val is not None:
            error = np.abs(original - upsampled)
            h, w = error.shape
            fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
            ax = plt.Axes(fig, [0., 0., 1., 1.])
            ax.set_axis_off()
            fig.add_axes(ax)
            ax.imshow(error, cmap='hot')
            plt.savefig(f'{output_folder}/pipeline_{kernel}_4_error.png', dpi=dpi)
            plt.close()
    
    print("\n" + "=" * 60)
    print("Image scaling completed!")
    print("=" * 60)

EXPERIMENT 1: Downscale only (test different factors)
Factor 2: (256, 256) → (128, 128)
Factor 4: (256, 256) → (64, 64)

EXPERIMENT 2: Upscale only (test different kernels)
Kernel k1: (256, 256) → (512, 512)
Kernel k2: (256, 256) → (512, 512)
Kernel k3: (256, 256) → (512, 512)

EXPERIMENT 3: Full pipeline (down then up) - Quality test

Kernel k1:
  Original: (256, 256)
  After downsample: (128, 128)
  After upsample: (256, 256)
  MSE: 842.1088

Kernel k2:
  Original: (256, 256)
  After downsample: (128, 128)
  After upsample: (256, 256)
  MSE: 180.3602

Kernel k3:
  Original: (256, 256)
  After downsample: (128, 128)
  After upsample: (256, 256)
  MSE: 180.3602

Image scaling completed!
